<a href="https://colab.research.google.com/github/PatronesGrupo4/Grupo4/blob/main/Challenges/Challenge_2__avance_Sergio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$\Large \textit{C0708 | Reconocimiento de Patrones}$$

$$\large \textbf{Challenge 2 | Métodos de aprendizaje supervisado: Regresión lineal, polinomial, lineal múltiple, KNN}$$

# Análisis exploratorio

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

df_infecciones = pd.read_csv('/content/drive/MyDrive/Patrones 2023.2/Semana 4/Challenge/DIAGNOSTICOS_POR_GRUPOS_DE_EDAD.csv', delimiter=';')
df_infecciones.head()

,PERIODO_DIAGNOSTICO,ESTABLECIMIENTO,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,FECHA_CORTE,SEXO,GRUPOS_POR_EDAD,DIAGNOSTICOS_CIE,CATEGORIA_DIAGNOSTICO,TOTAL_PACIENTES
0,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 25 A 29 AÑOS,INFECCIÓN POR SALMONELLA,OTRAS INFECCIONES DEBIDAS A SALMONELLA,1
1,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,MENORES DE 1 AÑO,OTRAS INFECCIONES INTESTINALES BACTERIANAS ESP...,OTRAS INFECCIONES INTESTINALES BACTERIANAS,1
2,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 60 A 64 AÑOS,ABSCESO AMEBIANO DEL PULMÓN,AMEBIASIS,2
3,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 65 AÑOS A MÁS,"AMEBIASIS, NO ESPECIFICADA",AMEBIASIS,1
4,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 5 A 9 AÑOS,GIARDIASIS [LAMBLIASIS],OTRAS ENFERMEDADES INTESTINALES DEBIDAS A PROT...,2


Descripcion general

In [12]:
print(df_infecciones.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6049 entries, 0 to 6048
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   PERIODO_DIAGNOSTICO    6049 non-null   int64 
 1   ESTABLECIMIENTO        6049 non-null   object
 2   DEPARTAMENTO           6049 non-null   object
 3   PROVINCIA              6049 non-null   object
 4   DISTRITO               6049 non-null   object
 5   UBIGEO                 6049 non-null   int64 
 6   FECHA_CORTE            6049 non-null   int64 
 7   SEXO                   6049 non-null   object
 8   GRUPOS_POR_EDAD        6049 non-null   object
 9   DIAGNOSTICOS_CIE       6049 non-null   object
 10  CATEGORIA_DIAGNOSTICO  6049 non-null   object
 11  TOTAL_PACIENTES        6049 non-null   int64 
dtypes: int64(4), object(8)
memory usage: 567.2+ KB
None


Descripcion estadistica

In [13]:
df_infecciones.describe()

,PERIODO_DIAGNOSTICO,UBIGEO,FECHA_CORTE,TOTAL_PACIENTES
count,6049.0,6049.0,6049.0,6049.000000
mean,202301.0,150101.0,20230615.0,2.610845
std,0.0,0.0,0.0,6.264612
min,202301.0,150101.0,20230615.0,1.000000
25%,202301.0,150101.0,20230615.0,1.000000
50%,202301.0,150101.0,20230615.0,1.000000
75%,202301.0,150101.0,20230615.0,2.000000
max,202301.0,150101.0,20230615.0,247.000000


# Encoding

Aplicamos ordinal encoding a las variables ordinales del grupo de edad

In [14]:
#import label encoder
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
#make an instance of Ordinal Encoder
encoder = OrdinalEncoder(categories=[['MENORES DE 1 AÑO', 'DE 1 A 4 AÑOS', 'DE 5 A 9 AÑOS', 'DE 10 A 14 AÑOS', 'DE 15 A 19 AÑOS', 'DE 20 A 24 AÑOS', 'DE 25 A 29 AÑOS', 'DE 30 A 34 AÑOS', 'DE 35 A 39 AÑOS', 'DE 40 A 44 AÑOS', 'DE 45 A 49 AÑOS', 'DE 50 A 54 AÑOS', 'DE 55 A 59 AÑOS', 'DE 60 A 64 AÑOS', 'DE 65 AÑOS A MÁS']])
df_infecciones_clean = df_infecciones
df_infecciones_clean['GRUPOS_EDAD_ENCODED'] = encoder.fit_transform(df_infecciones_clean[['GRUPOS_POR_EDAD']])
df_infecciones_clean.head()

,PERIODO_DIAGNOSTICO,ESTABLECIMIENTO,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,FECHA_CORTE,SEXO,GRUPOS_POR_EDAD,DIAGNOSTICOS_CIE,CATEGORIA_DIAGNOSTICO,TOTAL_PACIENTES,GRUPOS_EDAD_ENCODED
0,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 25 A 29 AÑOS,INFECCIÓN POR SALMONELLA,OTRAS INFECCIONES DEBIDAS A SALMONELLA,1,6.0
1,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,MENORES DE 1 AÑO,OTRAS INFECCIONES INTESTINALES BACTERIANAS ESP...,OTRAS INFECCIONES INTESTINALES BACTERIANAS,1,0.0
2,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 60 A 64 AÑOS,ABSCESO AMEBIANO DEL PULMÓN,AMEBIASIS,2,13.0
3,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 65 AÑOS A MÁS,"AMEBIASIS, NO ESPECIFICADA",AMEBIASIS,1,14.0
4,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 5 A 9 AÑOS,GIARDIASIS [LAMBLIASIS],OTRAS ENFERMEDADES INTESTINALES DEBIDAS A PROT...,2,2.0


Aplicamos label encoding a las variables nominales del diagnostico

In [15]:
#make an instance of Label Encoder
label_encoder = preprocessing.LabelEncoder()
df_infecciones_clean['CATEGORIA_DIAGNOSTICO_ENCODED'] = label_encoder.fit_transform(df_infecciones_clean['CATEGORIA_DIAGNOSTICO'])
df_infecciones_clean.head()

#escogemos de target el diagnostico y de feature importante el grupo por edad
x = df_infecciones_clean['GRUPOS_EDAD_ENCODED']
y = df_infecciones_clean['CATEGORIA_DIAGNOSTICO_ENCODED']

# Modelo de clasificacion KNN

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)

# Antes de aplicar el escalado, reformatea x_train y x_test
x_train = x_train.values.reshape(-1, 1)
x_test = x_test.values.reshape(-1, 1)

scaler=StandardScaler()

x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)



model=KNeighborsClassifier()
model.fit(x_train,y_train)

y_pred=model.predict(x_test)

Verificamos las metricas de calidad del modelo

In [18]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         6
          13       0.00      0.00      0.00         3
          16       0.00      0.00      0.00         2
          20       0.00      0.00      0.00         3
          22       0.00      0.00      0.00         1
          23       0.00      0.00      0.00         1
          24       0.00      0.00      0.00         1
          25       0.00      0.00      0.00         2
          26       0.00      0.00      0.00         3
          27       0.00      0.00      0.00         1
          28       0.00      0.00      0.00        12
          29       0.00      0.00      0.00         3
          31       0.00      0.00      0.00         5
          32       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.